In [65]:
import h5py
import pandas as pd
import numpy as np
from scipy import stats
import os
import json
from glob import glob
import zipfile
from collections import OrderedDict
from operator import itemgetter
from pga.return_pid import return_pid
from datetime import datetime
import numpy_indexed as npi
from tqdm import tqdm

In [33]:
os.chdir('D:/data/lineup_combos')
file = h5py.File('test_combo_5.h5', 'r')

file_keys = file.keys()

combos = np.array(file['batch_0'])
#combos = combos[1:,:5]

In [3]:
combos.shape

(98600005, 5)

In [4]:
#os.chdir('/home/valesco/Datasets/dk_downloads/')

sal_df = pd.read_csv('DKSalaries (36).csv', skiprows = 7, usecols = range(7,14))
sal_df.reset_index(inplace = True, drop = True)
sal_df.columns = ['pos', 'name_id', 'player_name', 'id', 'salary', 'tournament', 'team']
sal_df.head()
sal_df = sal_df.loc[sal_df['player_name'] != 'Stewart Hagestad']
sal_df = sal_df.loc[sal_df['player_name'] != 'Toto Gana']

In [5]:
pid_ls = []
for player in sal_df['player_name'].unique():
    pid = return_pid(player)
    
    if pid is not np.nan:
        pid_ls.append(pid)
    else:
        print(player, pid)
        
sal_df['pid'] = pid_ls        

In [58]:
sal_df['pid'] = sal_df['pid'].apply(lambda x: int(x))
sal_df.sort_values(by = 'pid', inplace = True)
sal_df['temp_pid'] = sal_df['pid'].rank(ascending = True)
sal_df['temp_pid'] = sal_df['temp_pid'].apply(lambda x: int(x - 1))
sal_df['sal/100'] = sal_df['salary'] / 100
sal_df['sal/100'] = sal_df['sal/100'].apply(lambda x: int(x))
sal_df['sal_rank'] = sal_df['salary'].rank(ascending = False)
sal_df['sal_rank'] = sal_df['sal_rank'].apply(lambda x: int(x))
sal_df.head()

,pos,name_id,player_name,id,salary,tournament,team,pid,temp_pid,sal/100,sal_rank
83,G,Fred Couples (8422982),Fred Couples,8422982,6300,Masters Tournament,Golf,1226,0,63,83
80,G,Bernhard Langer (8422959),Bernhard Langer,8422959,6400,Masters Tournament,Golf,1666,1,64,80
92,G,Sandy Lyle (8422962),Sandy Lyle,8422962,6000,Masters Tournament,Golf,1717,2,60,92
10,G,Phil Mickelson (8422942),Phil Mickelson,8422942,8700,Masters Tournament,Golf,1810,3,87,11
93,G,Larry Mize (8422966),Larry Mize,8422966,6000,Masters Tournament,Golf,1823,4,60,92


In [63]:
sal_dict = dict(zip(np.array(sal_df['temp_pid'].values).astype(np.int16), 
                    np.array(sal_df['sal/100'].values).astype(np.int16)))

top_df = sal_df.loc[sal_df['sal_rank'] <= 10]
rank_dict = dict(zip(np.array(top_df['temp_pid'].values).astype(np.int16),
                     np.array(top_df['sal/100'].values).astype(np.int16)))

In [49]:
#test_combos = npi.unique(combos[:-2999])
start = datetime.now()

combos = combos[1:,:5]
combos = npi.unique(combos)
combos = combos[np.where((combos < 93).all(axis = 1))]

sal_array = np.vectorize(sal_dict.__getitem__)(combos)

print(datetime.now() - start)

#test_combos[np.where((test_combos == (1,2,3,4,5,1)).all(axis = 1))]

NameError: name 'start' is not defined

In [132]:
final_array = np.zeros(shape = (1,6)).astype(np.int16)

for pid, salary in tqdm(rank_dict.items()):
    
    temp_sal = np.full((len(sal_array), 1), salary)

    sal_array = np.hstack((temp_sal, sal_array))
    indices = np.where((sal_array.sum(axis = 1) < 501) & (sal_array.sum(axis = 1) > 450))

    sal_array = sal_array[:,1:]

    temp_combo = combos[indices]
    temp_pid = np.full((len(temp_combo), 1), pid, np.int16)
    temp_combo = np.hstack((temp_pid, temp_combo))

    vals, counts = stats.mode(temp_combo, axis = 1)

    temp_array = np.hstack((temp_combo, counts)).astype(np.int16)
    temp_array = temp_array[temp_array[:,6] == 1]
    temp_array = temp_array[:,:-1]

    final_array = np.vstack((final_array, temp_array))

  0%|                                                                                           | 0/10 [00:00<?, ?it/s]C:\Users\vales\Anaconda3\lib\site-packages\numpy\core\numeric.py:301: FutureWarning: in the future, full((12795531, 1), 95) will return an array of dtype('int16')
  format(shape, fill_value, array(fill_value).dtype), FutureWarning)
 10%|████████▎                                                                          | 1/10 [00:20<03:06, 20.67s/it]C:\Users\vales\Anaconda3\lib\site-packages\numpy\core\numeric.py:301: FutureWarning: in the future, full((12795531, 1), 92) will return an array of dtype('int16')
  format(shape, fill_value, array(fill_value).dtype), FutureWarning)
 20%|████████████████▌                                                                  | 2/10 [00:38<02:39, 19.92s/it]C:\Users\vales\Anaconda3\lib\site-packages\numpy\core\numeric.py:301: FutureWarning: in the future, full((12795531, 1), 88) will return an array of dtype('int16')
  format(shape, 

In [95]:
sal_array[np.where((sal_array.sum(axis = 1) < 501) & (sal_array.sum(axis = 1) > 450))]

array([[  63.,   64.,  102.,  113.,  115.],
       [  63.,   64.,  106.,  113.,  115.],
       [  63.,   64.,  113.,   99.,  115.],
       ..., 
       [  60.,  113.,  115.,   82.,   86.],
       [  60.,   93.,   99.,   89.,  115.],
       [  60.,   93.,   99.,  115.,   86.]])

In [133]:
final_array.shape

(65311001, 6)

In [134]:
start = datetime.now()
test_sal_array = np.vectorize(sal_dict.__getitem__)(final_array[1:])
print(datetime.now() - start)

0:09:40.980470


In [143]:
sal_sum = test_sal_array.sum(axis = 1).reshape(len(test_sal_array), 1)
np.hstack((test_sal_array, sal_sum))

array([[ 95,  63,  64, ...,  87,  83, 452],
       [ 95,  63,  64, ...,  87,  92, 461],
       [ 95,  63,  64, ...,  87,  88, 457],
       ..., 
       [115,  60,  73, ...,  86,  62, 470],
       [115,  60,  73, ...,  62,  62, 458],
       [115,  60,  74, ...,  62,  62, 459]])

In [136]:
sal_sum.shape

(65311000,)

In [137]:
test_sal_array.shape

(65311000, 6)

In [142]:
sal_sum

array([451, 451, 451, ..., 500, 500, 500])

In [141]:
sal_sum.reshape(len(test_sal_array), 1)

array([[451],
       [451],
       [451],
       ..., 
       [500],
       [500],
       [500]])